# Read in the data

In [1]:
import pandas as pd
import numpy as np
import re
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]
data = {}
for i in range(0,len(data_files)):
    string="Projects/NYC School data/" +data_files[i]
    dummy=data_files[i].replace(".csv","")
    data[dummy]=pd.read_csv(string)
data["ap_2010"].head()

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
1,01M450,EAST SIDE COMMUNITY HS,19,21,s
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
4,02M296,High School of Hospitality Management,s,s,s


# Read in the surveys

In [11]:
all_survey = pd.read_csv("Projects/NYC School data/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pd.read_csv("Projects/NYC School data/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pd.concat([all_survey, d75_survey], sort=False, axis=0)

survey["DBN"] = survey["dbn"]

survey_fields = [
    "DBN", 
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
]
survey = survey.loc[:,survey_fields]
data["survey"] = survey

# Add DBN columns

In [3]:
data["hs_directory"]["DBN"]=data["hs_directory"]["dbn"]
#data["class_size"]["CSD"].dtype
data["class_size"]["padded_csd"]=data["class_size"]["CSD"].apply(lambda x:str(x).zfill(2))
data["class_size"]["DBN"]=data["class_size"]["padded_csd"]+data["class_size"]["SCHOOL CODE"]
data["class_size"]["DBN"].head()


0    01M015
1    01M015
2    01M015
3    01M015
4    01M015
Name: DBN, dtype: object

# Convert columns to numeric

In [4]:
cols=["SAT Math Avg. Score","SAT Critical Reading Avg. Score","SAT Writing Avg. Score"]
for i in cols:
    data["sat_results"][i]=pd.to_numeric(data["sat_results"][i],errors="coerce")
data["sat_results"]["sat_score"]=data["sat_results"][cols[0]]+data["sat_results"][cols[1]]+data["sat_results"][cols[2]]
data["sat_results"]["sat_score"].head()

def latitude(string):
    latlong=re.findall("\(.+\)",string)
    latlong[0]=latlong[0].replace("(","").replace(")","")
    latlong=latlong[0].split(",")
    return latlong[0]
def longitude(string):
    latlong=re.findall("\(.+\)",string)
    latlong[0]=latlong[0].replace("(","").replace(")","")
    latlong=latlong[0].split(",")
    return latlong[1]

data["hs_directory"]["lon"]=data["hs_directory"]["Location 1"].apply(longitude)
data["hs_directory"]["lat"]=data["hs_directory"]["Location 1"].apply(latitude)

data["hs_directory"]["lat"]=pd.to_numeric(data["hs_directory"]["lat"],errors="coerce")
data["hs_directory"]["lon"]=pd.to_numeric(data["hs_directory"]["lon"],errors="coerce")




# Condense datasets

In [5]:
class_size=data["class_size"]
class_size=class_size.loc[(class_size["GRADE "]=='09-12')&(class_size["PROGRAM TYPE"]=="GEN ED"),:]

class_size=class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
data["class_size"]=class_size

demographics=data["demographics"]
demographics=demographics.loc[demographics["schoolyear"]==20112012,:]
data["demographics"]=demographics

graduation=data["graduation"]
graduation=graduation.loc[(graduation["Demographic"]=='Total Cohort')&(graduation["Cohort"]=='2006'),:]
data["graduation"]=graduation


# Convert AP scores to numeric

In [6]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for col in cols:
    data["ap_2010"][col] = pd.to_numeric(data["ap_2010"][col], errors="coerce")

# Combine the datasets

In [7]:
combined = data["sat_results"]
combined=pd.merge(combined,data["ap_2010"],how="left",on="DBN")
combined=pd.merge(combined,data["graduation"],how="left",on="DBN")

for string in ["class_size","demographics","survey","hs_directory"]:
    combined=pd.merge(combined,data[string],how="inner",on="DBN")
combined = combined.fillna(combined.mean())
combined = combined.fillna(0)



# Add a school district column for mapping

In [8]:
combined["school_dist"]=combined["DBN"].apply(lambda x:x[0:2])
combined.head()


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,priority05,priority06,priority07,priority08,priority09,priority10,Location 1,lon,lat,school_dist
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,0,129.028846,197.038462,...,Then to New York City residents,0,0,0,0,0,"220 Henry Street\nNew York, NY 10002\n(40.7137...",-73.985260,40.713764,01
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.000000,49.000000,...,0,0,0,0,0,0,"200 Monroe Street\nNew York, NY 10002\n(40.712...",-73.984797,40.712332,01
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,EAST SIDE COMMUNITY HS,19.000000,21.000000,...,0,0,0,0,0,0,"420 East 12 Street\nNew York, NY 10009\n(40.72...",-73.983041,40.729783,01
3,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,0,129.028846,197.038462,...,0,0,0,0,0,0,"145 Stanton Street\nNew York, NY 10002\n(40.72...",-73.985673,40.720569,01
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522.0,574.0,525.0,1621.0,"NEW EXPLORATIONS SCI,TECH,MATH",255.000000,377.000000,...,0,0,0,0,0,0,"111 Columbia Street\nNew York, NY 10002\n(40.7...",-73.979426,40.718725,01


# Find correlations

In [12]:
correlations = combined.corr()
correlations = correlations["sat_score"]


# Plotting survey correlations

In [10]:
# Remove DBN since it's a unique identifier, not a useful numerical value for correlation.
survey_fields.remove("DBN")